https://simpletransformers.ai/docs/qa-minimal-start/ (Explicativo de web)


In [ ]:
!pip install -Uq wandb

In [ ]:
import wandb

# wandb.login()
wandb_project = "SimpleTransformers-QA"

In [ ]:
%%capture
!pip install simpletransformers

In [ ]:
import os
import requests
import json

In [ ]:
import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
url = "https://raw.githubusercontent.com/josuemzx/Chatbot-para-refugiados/main/data/Dataset%20propio/data.json"
data = json.loads(requests.get(url).text)
data = [item for topic in data['data'] for item in topic['paragraphs'] ]
   
#Dataset acutalizado

In [ ]:
data

In [ ]:
# Ejemplo de acceso a todos los registros del dataset:
data[0]['qas']

In [ ]:
# Ejemplo de acceso a una:
data[0]['qas'][0]['answers'][0]['text']

In [ ]:
# Ejemplo de acceso a un contexto:
data[0]['context']

In [ ]:
# Incluímos train_test_split

from sklearn.model_selection import train_test_split

y = data[0]['qas'][0]['answers'] # REVISAR (Consulta S): El tamaño de "y" es "1", entiendo que habría que asignar a "y" todos los valores de "answers" 
X = data[0]['context']

print("Tamaño 'y': ",len(y))
print("Tamaño 'X': ",len(X))


In [ ]:

X_train, y_train, X_test, y_test = train_test_split(X, y)

print('===== Train data =====')
print(X_train.shape)
print(X_train.head(2))
print('===== Test data =====')
print(X_test.shape)
print(X_test.head(2))

In [ ]:
train_data[0]

In [ ]:
url = "https://raw.githubusercontent.com/josuemzx/Chatbot-para-refugiados/main/data/Dataset%20propio/test_data.json"
test_data = json.loads(requests.get(url).text)
eval_data = [item for topic in test_data['test_data'] for item in topic['paragraphs'] ]

In [ ]:
eval_data[0]

In [ ]:
# Configure the model
model_args = QuestionAnsweringArgs(output_dir=" train4")
model_args.train_batch_size = 16
model_args.evaluate_during_training = True

model = QuestionAnsweringModel(
    "roberta", "IIC/roberta-base-spanish-sqac" , args=model_args
)


In [ ]:
# Train the model
model.train_model(train_data, eval_data=train_data)

In [ ]:
help(model.train_model)

In [ ]:
# Evaluate the model
result, texts = model.eval_model(eval_data)

In [ ]:
# Make predictions with the model
to_predict = [
    {
        "context":" Derecho a solicitar protección internacional.\n\n1. Las personas nacionales no comunitarias y las apátridas presentes en territorio español tienen derecho a solicitar protección internacional en España.\n\n2. Para su ejercicio, los solicitantes de protección internacional tendrán derecho a asistencia sanitaria y a asistencia jurídica gratuita, que se extenderá a la formalización de la solicitud y a toda la tramitación del procedimiento, y que se prestará en los términos previstos en la legislación española en esta materia, así como derecho a intérprete en los términos del artículo 22 de la Ley Orgánica 4/2000.\n\nLa asistencia jurídica referida en el párrafo anterior será preceptiva cuando las solicitudes se formalicen de acuerdo al procedimiento señalado en el artículo 21 de la presente Ley.\n\n3. La presentación de la solicitud conllevará la valoración de las circunstancias determinantes del reconocimiento de la condición de refugiado, así como de la concesión de la protección subsidiaria. De este extremo se informará en debida forma al solicitante.\n\n4. Toda información relativa al procedimiento, incluido el hecho de la presentación de la solicitud, tendrá carácter confidencial.",
         
        "qas": [
            {
                "question": "¿Puedo solicitar traductor?",
                "id": "0",
            }
        ],
    }
]

answers, probabilities = model.predict(to_predict)

print(answers)

In [ ]:
print(answers)

In [ ]:
# Make predictions with the model
to_predict = [
    {
        "context": "Las subvenciones concedidas a las ONG para la acogida y la integración de quienes conforman estos colectivos están financiadas con fondos que provienen de los presupuestos de la Secretaría General de Inmigración y Emigración, recibiendo cofinanciación, en algunos programas, del Fondo Social Europeo y del Fondo de Asilo, Migración e Integración.",
        "qas": [
            {
                "question": "donde solicito las subenciones?",
                "id": "0",
            }
        ],
    }
]

answers, probabilities = model.predict(to_predict)

print(answers)

Ojo, si ejecuta dos veces da error por los datos guardado